# 读取所有对比结果

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

In [ ]:
years = ['2020']
months = ['09']
output_dir = r'E:\\HLCourse\\experiment_marine_technology\\finally_project\\output\\'
dataframe_dir = r'E:\\HLCourse\\experiment_marine_technology\\finally_project\\dataframe\\'

In [ ]:
for yea in years:
    for mon in months:
        hy_dir_path = r'G:\\remote_sensing_data\\hy2b\\ALT\\'+ yea + '\\' + mon
        hyfiles = hyfiles + (glob.glob(hy_dir_path + '\*.nc'))
hyfiles.sort()